# Interactive TFX pipeline with eager execution

In [35]:
import os
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import example_gen_pb2
from tfx.types.artifact_utils import get_split_uri
from tfx.utils.dsl_utils import tfrecord_input
from tfx.components import ImportExampleGen
from tfx.components import StatisticsGen
from tfx.components import SchemaGen
from tfx.components import ExampleValidator
from tfx.components import Transform


In [36]:
HISTORY_SIZE = 5
DATA_PATH = './../data'
PIPELINE_PATH = './../interactive_pipeline'
PIPELINE_NAME = 'timeseries_forecasting'
TFRECORD_PATH = os.path.join(DATA_PATH, 'dataset.tfrecord')
TRANSFORM_MODULE = 'transform.py'

## Preprocessed data

This is output of the preprocessing step and input to the TFX pipeline (*dataset.tfrecord* file).

In [37]:
def read_sample(sample):
    features = {
        'sample': tf.io.FixedLenFeature((), tf.string),
        'label': tf.io.FixedLenFeature((), tf.float32)
    }
    example = tf.io.parse_single_example(sample, features)
    sample = tf.io.parse_tensor(example['sample'], out_type=tf.float32)
    sample = tf.reshape(sample, (HISTORY_SIZE, 7))
    label = example['label']
    return sample, label


raw_dataset = tf.data.TFRecordDataset([TFRECORD_PATH])
parsed_dataset = raw_dataset.map(read_sample)
for sample in parsed_dataset.take(1):
    print(f'Label: {sample[1].numpy()}')
    print(f'Sample: {sample[0].numpy()}')


Label: 0.2603398561477661
Sample: [[ -0.60067534  -2.9281392  -34.31954     -0.5931358   -1.018011
   -0.6294911   -1.1853565 ]
 [  3.0524564    0.98232865  -0.11870121   3.136877     0.2677397
    1.1172625    1.1052865 ]
 [  1.0651267   -0.33178836   0.10485055   1.0523465   -0.34572718
    0.36598098   1.2895724 ]
 [  0.16472098  -0.55382997   0.2547666    0.18104698  -0.48739374
    2.7555935   -0.83937824]
 [  1.101841    -0.34803745   0.31506872   1.0789726   -1.018011
    0.37893412   1.3018218 ]]


## TFX Pipeline

In [38]:
tf.executing_eagerly()
context = InteractiveContext(pipeline_name=PIPELINE_NAME,
                             pipeline_root=PIPELINE_PATH)


### ExampleGen Component

In [39]:
output_config = example_gen_pb2.Output(
        split_config=example_gen_pb2.SplitConfig(splits=[
            example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=4),
            example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
        ]))
examples = tfrecord_input(DATA_PATH)
example_gen = ImportExampleGen(input=examples,
                                output_config=output_config)

context.run(example_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: ./../interactive_pipeline/ImportExampleGen/examples/1, id: 2)]
        ))

Output of the ExampleGen Component:

In [40]:
examples_dir = example_gen.outputs['examples']._artifacts[0].uri
examples_path = os.path.join(examples_dir, 'train/data_tfrecord-00000-of-00001.gz')
raw_dataset = tf.data.TFRecordDataset([examples_path], compression_type='GZIP')

for raw_record in raw_dataset.take(1):
    print(repr(raw_record))

<tf.Tensor: shape=(), dtype=string, numpy=b'\n\xc2\x01\n\x11\n\x05label\x12\x08\x12\x06\n\x04DK\x85>\n\xac\x01\n\x06sample\x12\xa1\x01\n\x9e\x01\n\x9b\x01\x08\x01\x12\x08\x12\x02\x08\x05\x12\x02\x08\x07"\x8c\x01\xdc\xc5\x19\xbf\xa2f;\xc05G\t\xc2\xbf\xd7\x17\xbf/N\x82\xbfT&!\xbf\xc3\xb9\x97\xbfr[C@\xe4y{?\x9f\x19\xf3\xbd\x98\xc2H@.\x15\x89>u\x02\x8f?\x07z\x8d?\x12V\x88?*\xe0\xa9\xbe\xe2\xbb\xd6=J\xb3\x86?\'\x03\xb1\xbe\xdca\xbb>\xb5\x10\xa5?\x9e\xac(>\xcd\xc7\r\xbf\xc5p\x82>ad9>\xac\x8b\xf9\xbe\xa5[0@~\xe1V\xbf \t\x8d?\xf71\xb2\xbe\xb0P\xa1>\xc6\x1b\x8a?/N\x82\xbf\xa7\x03\xc2>\x19\xa2\xa6?'>


In [41]:
features_description = {
    'sample': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.float32)
}


def parse_example(raw_example):
    return tf.io.parse_single_example(raw_example, features_description)


parsed_dataset = raw_dataset.map(parse_example)

for parsed_record in parsed_dataset.take(1):
    print(tf.io.parse_tensor(parsed_record['sample'], out_type=tf.float32))
    print(parsed_record['label'])

tf.Tensor(
[[ -0.60067534  -2.9281392  -34.31954     -0.5931358   -1.018011
   -0.6294911   -1.1853565 ]
 [  3.0524564    0.98232865  -0.11870121   3.136877     0.2677397
    1.1172625    1.1052865 ]
 [  1.0651267   -0.33178836   0.10485055   1.0523465   -0.34572718
    0.36598098   1.2895724 ]
 [  0.16472098  -0.55382997   0.2547666    0.18104698  -0.48739374
    2.7555935   -0.83937824]
 [  1.101841    -0.34803745   0.31506872   1.0789726   -1.018011
    0.37893412   1.3018218 ]], shape=(5, 7), dtype=float32)
tf.Tensor(0.26033986, shape=(), dtype=float32)


### StatisticsGen Component

In [42]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

context.run(statistics_gen)

t be decoded as a UTF-8 string.


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(type_name: ExampleStatistics, uri: ./../interactive_pipeline/StatisticsGen/statistics/2, id: 3)]
        ))

Output of the StatisticsGen Component:

In [43]:
artifact_list = statistics_gen.outputs['output'].get()
train_artifacts_path = get_split_uri(artifact_list, 'train')
train_statistics_path = os.path.join(train_artifacts_path, 'stats_tfrecord')
eval_artifacts_path = get_split_uri(artifact_list, 'eval')
eval_statistics_path = os.path.join(eval_artifacts_path, 'stats_tfrecord')

In [44]:
train_statistics = tfdv.load_statistics(train_statistics_path)
eval_statistics = tfdv.load_statistics(eval_statistics_path)

tfdv.visualize_statistics(lhs_statistics=eval_statistics,
                          rhs_statistics=train_statistics,
                          lhs_name='Eval',
                          rhs_name='Train')

### SchemaGen Component

In [45]:
schema_gen = schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'],
                                    infer_feature_shape=True)

context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(type_name: Schema, uri: ./../interactive_pipeline/SchemaGen/schema/3, id: 4)]
        ))

Output of the SchemaGen Component:

In [46]:
schema_directory = schema_gen.outputs['output'].get()[0].uri
schema_path = os.path.join(schema_directory, 'schema.pbtxt')

schema = tfdv.load_schema_text(schema_path)
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'sample',BYTES,required,,-
'label',FLOAT,required,,-


### ExampleValidator Component

In [47]:
example_validator = ExampleValidator(statistics=statistics_gen.outputs['statistics'],
                                     schema=schema_gen.outputs['schema'])

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(type_name: ExampleAnomalies, uri: ./../interactive_pipeline/ExampleValidator/anomalies/4, id: 5)]
        ))

Output of the ExampleValidator Component:

In [48]:
validator_directory = example_validator.outputs['output'].get()[0].uri
validator_path = os.path.join(validator_directory, 'anomalies.pbtxt')

anomalies = tfdv.load_anomalies_text(validator_path)
tfdv.display_anomalies(anomalies)

### Transform Component

In [49]:
transform = Transform(examples=example_gen.outputs['examples'],
                      schema=schema_gen.outputs['schema'],
                      module_file=TRANSFORM_MODULE)

context.run(transform)

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./../interactive_pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/9d53e7a08c2c4a7eba93dcbfaabd00fc/saved_model.pb
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: ./../interactive_pipeline/Transform/transform_graph/5, id: 6)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: ./../interactive_pipeline/Transform/transformed_examples/5, id: 7)]
        ))

Output of the Transform Component:

In [51]:
transform_directory = transform.outputs['transformed_examples'].get()[0].uri
transform_path = os.path.join(transform_directory, 'train', 'transformed_examples-00000-of-00001.gz')

transformed_dataset = tf.data.TFRecordDataset([transform_path], compression_type='GZIP')

for transformed_record in transformed_dataset.take(1):
    print(repr(transformed_record))

<tf.Tensor: shape=(), dtype=string, numpy=b"\n\xb3\x01\n\x11\n\x05label\x12\x08\x12\x06\n\x04DK\x85>\n\x9d\x01\n\x06sample\x12\x92\x01\x12\x8f\x01\n\x8c\x01\xdc\xc5\x19\xbf\xa2f;\xc05G\t\xc2\xbf\xd7\x17\xbf/N\x82\xbfT&!\xbf\xc3\xb9\x97\xbfr[C@\xe4y{?\x9f\x19\xf3\xbd\x98\xc2H@.\x15\x89>u\x02\x8f?\x07z\x8d?\x12V\x88?*\xe0\xa9\xbe\xe2\xbb\xd6=J\xb3\x86?'\x03\xb1\xbe\xdca\xbb>\xb5\x10\xa5?\x9e\xac(>\xcd\xc7\r\xbf\xc5p\x82>ad9>\xac\x8b\xf9\xbe\xa5[0@~\xe1V\xbf \t\x8d?\xf71\xb2\xbe\xb0P\xa1>\xc6\x1b\x8a?/N\x82\xbf\xa7\x03\xc2>\x19\xa2\xa6?">


In [54]:
transformed_features_description = {
    'sample': tf.io.FixedLenFeature([HISTORY_SIZE, 7], tf.float32),
    'label': tf.io.FixedLenFeature([], tf.float32)
}


def parse_transformed_example(example):
    return tf.io.parse_single_example(example, transformed_features_description)


parsed_dataset = transformed_dataset.map(parse_transformed_example)

for parsed_record in parsed_dataset.take(1):
    print(parsed_record['sample'])
    print(parsed_record['label'])

tf.Tensor(
[[ -0.60067534  -2.9281392  -34.31954     -0.5931358   -1.018011
   -0.6294911   -1.1853565 ]
 [  3.0524564    0.98232865  -0.11870121   3.136877     0.2677397
    1.1172625    1.1052865 ]
 [  1.0651267   -0.33178836   0.10485055   1.0523465   -0.34572718
    0.36598098   1.2895724 ]
 [  0.16472098  -0.55382997   0.2547666    0.18104698  -0.48739374
    2.7555935   -0.83937824]
 [  1.101841    -0.34803745   0.31506872   1.0789726   -1.018011
    0.37893412   1.3018218 ]], shape=(5, 7), dtype=float32)
tf.Tensor(0.26033986, shape=(), dtype=float32)
